In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
from google.colab import drive
drive.mount('./MyDrive')

Mounted at ./MyDrive


In [ ]:
cd MyDrive/My Drive

/content/MyDrive/My Drive


In [11]:
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import reuters
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

plt.rcParams['font.family'] = 'Malgun Gothic'

In [2]:
# train, test 분리
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=1000, test_split=0.2)

# 패딩
x_train = pad_sequences(x_train, maxlen=100)
x_test = pad_sequences(x_test, maxlen=100)

2113536/2110848 [==============================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/reuters.py:143: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/reuters.py:144: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [3]:
scaler = MinMaxScaler()
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [4]:
model = SVC(kernel='rbf', gamma=0.1, C=0.1)
model.fit(x_train, y_train)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [5]:
predicted_svm = model.predict(x_test)

In [6]:
print('정확도 = %.3f ' % model.score(x_test, y_test))

정확도 = 0.362 


In [7]:
param_grid = {
    'C': [0.01, 0.1, 1.0, 10.0],
    'gamma': [0.01, 0.1, 1.0, 10.0]
}

svc_model = SVC(kernel='rbf')
gs_svc = GridSearchCV(estimator=svc_model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=1)

gs_svc.fit(x_train, y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=1,
             param_grid={'C': [0.01, 0.1, 1.0, 10.0],
                         'gamma': [0.01, 0.1, 1.0, 10.0]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [8]:
print('최적 하이퍼 파라미터: ', gs_svc.best_params_)
print('최적 정확도: {:.3f}'.format(gs_svc.best_score_))

최적 하이퍼 파라미터:  {'C': 10.0, 'gamma': 0.01}
최적 정확도: 0.371


In [12]:
# gridsearch로 찾은 최적 하이퍼파라미터 사용
gs_svc_new = SVC(kernel='rbf', C=10.0, gamma=0.01)
gs_svc_new.fit(x_train, y_train)
pred = gs_svc_new.predict(x_test)

print('accuracy: {:.3f}'.format(accuracy_score(y_test, pred)))
print('Precision: {:0.3f}'.format(precision_score(y_test, pred, average='macro')))
print('recall: {:0.3f}'.format(recall_score(y_test, pred, average='macro')))

accuracy: 0.390
Precision: 0.281
recall: 0.086


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
